In [1]:
from pathlib import Path
from keras.preprocessing import image
import os
import shutil
from keras.utils import np_utils
import pandas as pd
from keras.models import load_model

Using TensorFlow backend.


## Data Preprocessing

In [2]:
data=pd.read_csv("labels.csv")
df=data.set_index('id')

In [3]:
'''p=Path("./test/")
target='datasets/test/'
list_img=[]

for i ,j in enumerate(p.glob("*.jpg")):
    idx=str(j)[3:-4]
    name=df.loc[idx][0]
    final_target=target+name
    if not os.path.exists(final_target):
        os.makedirs(final_target)
    src=os.path.join('test',str(j)[6:])
    dest=os.path.join(final_target,str(j)[6:])
    shutil.copy2(src,dest)
'''
    

'p=Path("./test/")\ntarget=\'datasets/test/\'\nlist_img=[]\n\nfor i ,j in enumerate(p.glob("*.jpg")):\n    idx=str(j)[3:-4]\n    name=df.loc[idx][0]\n    final_target=target+name\n    if not os.path.exists(final_target):\n        os.makedirs(final_target)\n    src=os.path.join(\'test\',str(j)[6:])\n    dest=os.path.join(final_target,str(j)[6:])\n    shutil.copy2(src,dest)\n'

In [4]:
folders=os.listdir('datasets/train/')

In [5]:
img_to_dict={}
images=[]
labels=[]
for i,f in enumerate(folders):
    img_to_dict[f]=i

In [6]:
for folder_name in folders:
    path=os.path.join('datasets/train/',folder_name)
    for imgs in os.listdir(path):
        img=image.load_img(os.path.join(path,imgs),target_size=((224,224)))
        imgArray=image.img_to_array(img)
        images.append(imgArray)
        labels.append(img_to_dict[folder_name])

In [7]:
import numpy as np
from numpy import random

In [8]:
combined=list(zip(images,labels))
random.shuffle(combined)

images[:],labels=zip(*combined)

In [9]:
X_train=np.array(images)
labels=np.array(labels)

### Y_train to one_hot_vector

In [10]:
Y_train=np_utils.to_categorical(labels)

### Creating neural network with resnet50

In [11]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model
from matplotlib import pyplot as plt

In [12]:
model=ResNet50(include_top=False,weights='imagenet',input_shape=((224,224,3)))

/home/amit/anaconda3/envs/deep2/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [13]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [14]:
av1=GlobalAveragePooling2D()(model.output)
fc1=Dense(256,activation="relu")(av1)
d1=Dropout((0.5))(fc1)

fc2=Dense(10,activation="softmax")(d1)

model_new=Model(input=model.input,output=fc2)
model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

/home/amit/anaconda3/envs/deep2/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


In [15]:
adam=Adam(lr=0.00003)
model_new.compile(loss="categorical_crossentropy",optimizer=adam,metrics=["accuracy"])

In [16]:
for ix in range(169):
    model_new.layers[ix].trainable=False

In [17]:
model_new.compile(loss="categorical_crossentropy",optimizer=adam,metrics=["accuracy"])
model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [18]:
hist=model_new.fit(X_train,Y_train,shuffle=True,batch_size=16,epochs=10,validation_split=0.2)

#### saving model 

In [81]:
model_new.save("mod.h5")

In [22]:
model_new=load_model("mod.h5")

In [23]:
folders=os.listdir('datasets/test/')
img_to_dict_test={}
images_test=[]
labels_test=[]
for i,f in enumerate(folders):
    img_to_dict_test[f]=i

In [24]:
for folder_name in folders:
    path=os.path.join('datasets/test/',folder_name)
    for imgs in os.listdir(path):
        img=image.load_img(os.path.join(path,imgs),target_size=((224,224)))
        imgArray=image.img_to_array(img)
        images_test.append(imgArray)
        labels_test.append(img_to_dict[folder_name])

In [25]:
X_test=np.array(images_test)
y_test=np.array(labels_test)
Y_test=np_utils.to_categorical(y_test)

### Test Accuracy

In [26]:
test_data=model_new.evaluate(X_test,Y_test)

20/20 [==============================] - 7s 375ms/step


In [27]:
print("Test Accuracy: ",test_data[1])

Test Accuracy:  0.8999999761581421


#### Loading Model

In [28]:
folders=os.listdir('datasets/train/')
img_to_dict={}
img_to_label={}
images=[]
labels=[]
for i,f in enumerate(folders):
    img_to_dict[i]=f
    img_to_label[f]=i

In [29]:
def prediction(img):
        img=image.load_img(img,target_size=((224,224)))
        imgFinal=image.img_to_array(img)
        imag=np.array(imgFinal)
        imag=imag.reshape((-1,224,224,3))
        model=load_model("mod.h5")
        p=model.predict(imag)
        s=np.max(p)
        i=p=np.argmax(p)
        return s,img_to_dict[i]

In [30]:
folders=os.listdir('datasets/test/')
y_act=[]
y_pred=[]
y_score=[]
cnt=0
for f in folders:
    print(f)
    path=os.path.join('datasets/test/',f)
    for img in os.listdir(path):
        y_act.append(img_to_label[f])
        cnt=cnt+1
        print(cnt)
        path2=os.path.join(path,img)
        s,p=prediction(path2)
        y_score.append(s)
        y_pred.append(img_to_label[p])

tibetan_mastiff
1
2
pug
3
4
malamute
5
6
chihuahua
7
8
doberman
9
10
french_bulldog
11
12
golden_retriever
13
14
saint_bernard
15
16
beagle
17
18
scottish_deerhound
19
20


In [31]:
from sklearn.metrics import confusion_matrix,f1_score,roc_auc_score,accuracy_score

#### F1 Score

In [32]:
f1_score_result=f1_score(y_act,y_pred,average="macro")

In [33]:
print(f1_score_result)

0.8933333333333333


#### Confusion_metrics

In [34]:
confusion_metrics_result=confusion_matrix(y_act,y_pred)

In [35]:
print(confusion_metrics_result)

[[1 0 0 0 0 0 0 1 0 0]
 [0 2 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 2]]
